# 为超过Token长度限制的文本创建摘要

在很多场景下我们都需要处理超过 4096 个 token 的文本，譬如文章、书籍、电影字幕等。当然我们总是有 token 容量更大的选择，譬如 GPT-4 32K；但是既然存在上限，那么就总有超限的时候。为了处理这种情况，我们采取逐步压缩的方案。

## 思路

1. 将要处理的文本分段，每一段不超过 token 上限。
2. 将每一段文本分别生成摘要，并存储摘要后的内容。
3. 处理完所有分段后，汇总摘要后的文本，如果文本总量仍然超限，那么重复步骤 1 和 2。
4. 如果摘要后的文本已经不再超限，那么直接生成最终摘要。

需要注意的是，这样的处理方法会导致信息丢失。

In [44]:
import os
import openai
import tiktoken

GPT_MODEL = 'gpt-3.5-turbo'
MAX_TOKENS = 3000

openai.api_key = os.getenv('OPENAI_API_KEY')
openai.api_base = os.getenv('OPENAI_API_BASE')

def num_tokens(text: str, model: str = GPT_MODEL) -> int:
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))

## 准备要处理的文本

这里我们直接硬编码一篇文章，然后用上面定义的 `num_tokens` 函数来计算 token 数量。

In [6]:
long_text = '''36氪专访 | 亚马逊云科技顾凡：云计算、大数据和人工智能是加速医疗与生命科学行业数字化转型的重要途径
Ben
·2023-05-16 13:42
关注
加速推进创新成果从实验室到真实世界的落地
“在1990年，需要十三年才能完成一个人基因测序，现在只需要一天就能测60个人，而每个人的测序成本降到百万分之一，而大规模的基因测序，需要调用数万台虚拟服务器同时进行。”

“十年间，亚马逊云科技服务了全球超过4200个医疗及生命科学行业客户，包括辉瑞、拜尔、罗氏、默沙东、GE、飞利浦等头部企业。全球前十大药企中，九家都在使用亚马逊云科技。”

“亚马逊云科技通过了全球百项安全合规认证，符合全球50+医疗生命科学合规要求。”

“20 多年来，人工智能和机器学习一直是亚马逊关注的焦点。亚马逊云科技的使命是让各种技能水平的开发人员和各种规模的组织都有机会使用生成式AI进行创新。”

4月27日，在“2023亚马逊云科技医疗与生命科学行业峰会”上，亚马逊云科技大中华区战略业务发展部总经理顾凡在演讲中围绕“数据、算力和体验”正在加速行业数字化创新，重点介绍了亚马逊云科技在全球以及中国医疗与生命科学行业的实践。

在会后的座谈中，顾凡和亚马逊云科技大中华区医疗及生命科学行业总监黄庆春，共同与36氪及其他参会者进行了深入地交流。

对于目前大热的AI大模型，顾凡认为：“我们既不应该去高估AIGC大模型的力量，但是更重要的是，不能低估AIGC大模型会重塑大量行业客户的体验。过去20多年的人工智能和机器学习一直都是亚马逊关注的焦点，亚马逊无论是对外的服务，还是对内部的运营，到处都可以看到人工智能和机器学习的身影。在生成式AI方面，我们的使命就是，要让各种技能水平的开发人员和各种组织都有机会使用生成式AI进行创新。”

亚马逊云科技长期关注生物制药、基因测序和医疗保健领域的客户需求，发现三大共同挑战：

数据挑战：数据量的剧增带来洞察力挑战，科学家发现，自然界里可能成为药物的分子个数有10的60次方，而太阳系的原子个数才是10的50次方；
算力挑战：在1990年，需要十三年才能完成一个人基因测序，现在只需要一天就能测60个人，而每个人的测序成本降到百万分之一，而大规模的基因测序，需要调用数万台虚拟服务器同时进行；
体验挑战：医院用的医疗信息软件有近100种，常用的就有约20，医疗机构需要适合行业用户的解决方案，不需要有专业的计算机背景也可以使用开箱即用的解决方案支持业务。
医疗与生命科学领域的数据挑战尤为突出，为此，亚马逊云科技提供了基因测序领域的数据全生命周期管理解决方案, 该解决方案包括数据上传、数据分析、数据共享和数据备份及存档全生命周期。

Amazon S3对象存储提供高达八层的存储层级，提供的自动分层功能可以更高效地存储数据和实现成本节约。同时亚马逊云科技还将自动分层功能扩展至Amazon Elastic File System (Amazon EFS)云原生文件存储之中，进一步降低数据管理的难度。

亚马逊云科技在2022年re:Invent期间最新推出了Amazon Omics服务，该服务能够帮助行业用户更快速地进行基因分析研究，从而加速新药研发、疾病诊断和治疗等领域的进展。

晶泰科技是一家智能化自动化驱动的药物研发科技公司，首创了智能计算、自动化实验和专家经验相结合的药物研发新模式，利用人工智能、自动化等前沿技术，显著缩短药物研发周期，提高药物研发成功率。目前支撑晶泰科技药物研发业务的计算调度平台已经构建在亚马逊云科技之上，每年可以节省大量的运营成本。

海量数据的计算不断推高算力需求，亚马逊云科技打造了一系列的创新服务，助力客户打造极致性能的云上HPC集群以加速新药研发：

针对HPC优化的Amazon EC2计算实例；
EFA网络提供具有高达400Gbps的网络带宽的节点间通信，以连接 AmazonEC2 实例网络；
亚毫秒级的高性能文件系统 Amazon FSx for Lustre 快速处理大规模海量数据；
集群调度管理工具Amazon Parallel Cluster可简化HPC集群的部署和管理；
云原生批处理调度器Amazon Batch 支持跨亚马逊云科技 计算服务和功能扩展数十万个计算任务。
在更前沿的科技领域，亚马逊云科技构建了完整的量子计算生态系统，包括量子服务Amazon Bracket、量子实验室以及量子计算中心。可提供了一键部署的量子计算/经典计算混合架构，通过Amazon Braket量子计算平台调用量子计算资源进行实验，为量子计算在药物发现方面的应用探索新思路。

医疗和生命科学行业的用户，更加青睐即开即用的解决方案，为此，亚马逊云科技专门构建的行业解决方案包括：

Amazon Omics，用于存储、分析和阐述基因组学、转录组学和其他组学数据的托管服务；
Amazon HealthLake Imaging医疗成像服务，它可以在云中以PB级规模存储、访问和分析医学图像，降低存储成本，提供亚秒级图像检索和自动化基础设施管理；
Amazon Comprehend Medical，是一项符合 HIPAA 要求的自然语言处理 (NLP) 服务，它使用经过预先训练的机器学习理解和提取医学文本中的健康数据，如处方、程序或诊断。
亚马逊云科技在大模型与医疗及生命科学结合的探索也已经有序推进，不久前，飞利浦医疗宣布与亚马逊云科技合作，将飞利浦医学影像系统Philips HealthSuite Imaging部署在云端，并且使用Amazon Bedrock快速开发生成式人工智能应用程序，如医护人员的语音识别，同时生成专业诊断信息，降低模型开发成本，而不是从头构建LLM基础模型，还能够确保数据安全和隐私保护。

亚马逊云科技还开放了96种生命科学公开数据集，涵盖生命科学、基因、肿瘤以及新冠等重要领域。

以下为36氪等媒体与亚马逊云科技大中华区战略业务发展部总经理顾凡的访谈，内容经摘编：
媒体：亚马逊云科技如何帮助垂直领域的客户将其私有数据在第三方大模型上训练？
顾凡：客户的需求在于想用大模型去改进客户体验或者项目，而大模型之所以成功，是要抓取非常多数据去反复学习。难点在于，有些第三方大模型是闭源的，用户只能送数据，而无法训练。

Amazon Bedrock对接了亚马逊自己的大模型，以及一些第三方大模型，从技术上、架构上让用户能够调用大模型与私有数据结合，去训练自己的模型，其解决的核心问题是行业中最后一公里的精度。

举例来说，一个企业有40%-50%的问题可以靠机器人来解决，解决不了的部分需要依靠大模型和大量数据做定制化，Amazon Bedrock要做的事情就是帮助客户结合大模型来解决业务中的具体问题。

媒体：企业普遍担心数据安全问题？以及训练成本是否会很高？
顾凡：亚马逊云科技上的账号对应独立的VPC，VPC（Virtual Private Cloud）就好像云上的隔离网络，在VPC中的数据，只有自己的ID能够访问。也就是说在VPC总去调用Amazon Bedrock的API、以及VPC中的数据，去完成联合训练，最终定制化模型的API也只有自己能使用，所以没有人能够使用VPC中的数据。

Amazon Bedrock还在试用阶段，成本还不是主要问题。这个行业在快速演进迭代，要想走向普适化，就得把成本拉下来。早期我们会计算在ChatGPT API的成本、在Amazon Bedrock API的成本，长线来看AI要普适化，成本是必须要解决的问题。

媒体：目前大模型还要解决哪些方向上的问题？
顾凡：一方面是如何解决客户的核心诉求，就是定制化数据训练问题；另一方面，Amazon Bedrock一定是走开放的路线，所以会接入其他的大模型，以不断优化成本。更重要的是，开源和客户商业化应用之间的距离，需要定制化联合训练。有些是易用性问题，可以由云厂商去解决；还有一些会和底层算力密切相关，不但需要GPU，还要看有没有能力自研芯片，比如亚马逊云科技自研的Trainium和Inferentia芯片追求的就是最极致性价比。

媒体：亚马逊云科技96种生命科学公开数据集的数据来源、以及数据集的构建过程是什么样的？开放过程是公益性质的么？
黄庆春：The Registry Open Data on Amazon Web Services （RODA）公开数据集是亚马逊云科技一个全球项目，与数十家科研机构来合作。数据集由这些科研机构提供的，托管在亚马逊云科技，愿意为科学家们免费开放使用。

现在亚马逊云科技把这个数据集与内部很多数据分析功能结合，科学家不仅可以接触到这些公开数据集，还可以在上面直接利用亚马逊云科技的数据分析工具，以及人工智能服务，在公开数据集上进行加工，获得所需的洞察。

整个公开数据集是不收费的，但是因为公开数据集是由各个不同的科研机构提供，所以有不同的授权机制，有些个人机构不允许作为商用。因为不同的数据集属于不同的拥有方，登录的时候要点选一些同意或不同意的条款，之后就可以免费使用。

媒体：在医疗健康领域如何保证数据在深度学习之后的数据安全性？如何避免造成误判、误诊的情况？
顾凡：生成式AI在医疗和生命科学行业中还在探讨阶段，我们既不应该高估它，也不应该低估它。低估可能竞争对手会先蹚出一条路，但高估它盲目地蹭热点是没有任何意义的，因为它的应用场景无论是为患者，还是为降本增效，必须能达到目的才有生命力。

大模型和生成式AI最终的目的是普惠，让用户不必去探讨AIGC工具，而投入更多精力去探讨场景。包括我们分享的飞利浦案例，都还在一个探讨阶段，但是探讨阶段的核心在于要么能够提供客户体验，要么能够降本增效。

数据安全中很重要的首先是要保护好客户的数据，其中包含了大量患者信息。所以一方面基础层要合规，在合规的基础上，AIGC要解决刚才探讨的矛盾：又想用大模型，但是又需要保护自己的私有数据，同时还想用自己的私有数据训练定制化模型，这就要靠云的技术去解决。

黄庆春：亚马逊云科技在全球遵守超过50种专门针对医疗行业的合规要求，像美国的HIPAA等，这是一个底线，先要符合行业的安全合规，再去拓展相关应用。

现在的探索我们称之为辅助阶段，现在有很多试点在做，比如新药研发阶段，我们用大模型去阅读海量文献和过去的实验，给未来要做的实验提出指导性意见。但是最后是否采取这个实验方法，还是由化学家、药学家做决定。在医学上，我们会阅读大量电子病历或其他数据，去看病人整个疾病的发展周期，但是最后也是给出建议，由主治医生决定诊疗方式是否适用。现阶段AI不是替换医生，而是培养懂AI的医生。

顾凡：其他行业也有同样的问题，有人会担心AI会不会替代很多人，实际上要看这些人会不会用AI。比如最经典的AIGC应用场景是在客服中心，作为客户找客服的时候，通常还是希望真人来处理问题。

媒体：新药的研发往往需要10年甚至更长时间，云技术的应用在生命科学行业能解决哪些问题和痛点？
顾凡： 一款新药可能用10年时间，20亿美金，才有10%的成功率。国家大力主张国内的新药研发一定要跟上，而不是仿制药。显然策也有，挑战也有。10%的成功率就意味着，创业风险太高，一定要检查长期主义。

云计算作为数字化底座，真正能在这个行业解决的问题帮客户更快，以及帮客户获得高性价比，以及帮客户更精准地发现适合的化合物。

所以最重要的三个方向是：高性能计算，AI辅助药物研发以及量子计算。这三个万变不离其宗，算力技术的演进迭代中，最初计算机辅助药物研究，高性能计算用得比较多，当AI出现之后，很多人更多思考，到底AI辅助药物研发会带给药物辅助研发哪些想象空间。

现在，我们的客户在量子计算方面已经取得了成功，未来可能算法是跨经典计算和量子计算的两种计算资源，在这种混合结构更有机会破局，让现在研发的计算速度大幅提升。

算力的发展会帮客户以云计算作为数字化底座，让研究药物的科学家能够随时访问、调用数据。让他们专注于自己的专业领域，而不是花时间去研究IT流程，这就是云厂商的价值。亚马逊云科技把量子计算等最复杂的科技变成API，使药物研发出现跨跃式的性能提升。

媒体：如何使AI技术人才和现实中研发需求达到匹配？在实际的应用中，AI制药还是在老药新用方面做得比较多，有没有可能AI也能设计出一些新药物？
黄庆春：学医和学AI互相不太理解确实是市场现状，这也是为什么亚马逊云科技要做从数据存储到算力，再到行业解决方案，我们要降低AI使用门槛。门槛降低到让没有很强计算机背景的人，可以快速使用AI的成果，实现普适化的AI。亚马逊云科技既提供云原生应用给计算机水平比较高的开发者自己搭建新应用，也提供指导方案给有一定计算机水平的初阶开发人员，还会提供端到端、开箱即用的解决方案。亚马逊云科技在用不同的解决方案降低AI的门槛。

顾凡：客户希望自己构建AI辅助药物研发平台，亚马逊云科技可以提供Amazon SageMaker，帮客户把底层的机器学习、人工智能工作流、基础资源全部管起来，同时也会把一些常用算法和Amazon SageMaker打包，作为方案指导，让客户快速去实验。

黄庆春：计算机辅助药物研发90年代就有了，发展到21世纪以后AI越来越强，AI监督学习的时候，是利用历史数据训练模型，一些未解的问题，比如老药有没有新疗效，或者化合物适不适合做先导化合物。

现在业界更希望干湿实验结合，人工智能辅助药物发现AIDD(AI DrugDesign)称为干实验，就是用计算机做实验，没有实验器材，业界还希望保留一定的湿实验，但干实验的好处是可以改善湿实验成本和进程。举例来说很多客户表示AIDD可以筛查掉可能不适合做药的化合物，这样的话需要做的湿实验数量就降低了很多。药界的观点是AIDD是很好技术，但还是要配合传统药物研发的技术来用。随着技术发展，可能未来AI覆盖的领域越来越多，湿实验可能越来越少。

媒体：综合医药研发的成本、风险、以及投入，中国客户与国外客户有哪些差异？
黄庆春：有一些差异，但主要还是共性，从行业需求出发，国内市场现状使AIDD等年限相对短一点。国外最近一家AI上市公司薛定谔（Schrodinger），在2000年初就成立了。国内很多是在近五年、十年。亚马逊云科技不只是一个技术提供方，还有对初创企业的云创计划，也有IDAC（数字化赋能中心），以及合作伙伴计划，和应用市场Marketplace。IDAC提供的不单是技术赋能，还提供生态赋能、能力赋能等其他一系列的赋能。

比如英矽智能刚成立就与亚马逊云科技合作，可以称为云原生的人工智能企业，它确实得到了很大的进展，首先是研发费用，英矽最近推出了第一款治疗肺纤维化的药物，只用了200多万美金就做出来了。英矽是一个非常有行业特点的公司，它的全流程都与亚马逊云科技合作。做出一个药，除了要研发化合物，还要做临床试验，这些英矽都采用了亚马逊云科技相关的技术，也是全球第一个推出了由AI为主要技术的药物。

媒体：有没有成本对比分析，比如通过干实验能够在哪些领域降低湿实验的成本？干实验和湿实验这两者的成本，在医药行业怎么去做对比分析？
黄庆春：首先干实验和湿实验不是竞争关系。传统的湿实验也用数字化技术，比如LIMS（Laboratory Information Management System）和ELM。干实验可能更多用数字化技术，俗称用计算机来模拟实验。湿实验现在亚马逊云科技也有数字化实验室解决方案，利用数字化技术去提升传统干实验中，比如设备之间的数据采集、数据分析。云计算能够帮助干实验和湿实验提高效率，当然成本也会更低。

医药行业要降低试错成本，提升成功的快速弹性。云计算有个特性是按需付费，如果自己搭建一个本地数据中心，可能是90%的失败率，但是用云的话，虽然还是90%的失败率，但是计算成本、存储成本都大大降低了，其实是很大地降低了试错成本。

另外在药物研发中，如果发现了一个潜在药物特别有效，要越快越好，但还有一个挑战是10年。我们有另外一个案例，新冠疫情早期Moderna数十天就把新冠疫苗推向上市，Moderna是亚马逊云科技全球客户，无论实验还是生产，大部分应用都放在亚马逊云科技上。所以除了可以把失败成本压到很低之外，还有一方面是成功以后可以加速上市。

媒体：云计算和生命科学相结合进行研究，会产生特别庞大的数据量，如何在海量数据中快速找到所需要的数据？
顾凡：数据是今天探讨话题的核心，从数据存储来说，医疗生命科学行业的数据比互联网公司的数据要大很多。对于生命科学医疗行业，首先要解决的是先把它存下来，从存的角度，Amazon S3天然就有8层存储，放在热的层，最关键是及时调用，然后计算速度、分析速度要快，放在冷里面是暂时用不到，但使用时，能马调出来。

亚马逊云科技做的第一件事，是行业中的专家要把最佳实践总结下来，把这些最佳实践结合亚马逊Amazon S3云服务，打成一个包，叫方案指南，下游客户拿着方案指南能马上知道基因数据怎么存更有效。另外像今天介绍的Amazon Omics，就是专门针对医疗行业推出的托管服务，它的好处就是用户不用学，Amazon Omics可以智能分层。

数据本身有生命线，存完后一定要分析。从分析的角度来说，亚马逊云科技除了有非常多的数据分析的工具之外，专门针对医疗行业还有两个分析工具，一个是Amazon Omics，专门针对基因数据内嵌了基因分析的算法，和亚马逊云科技云上分析的工具已经打通，客户一站式使用的易用性体验就更强。

另外一个是Amazon HealthLake，是一个非常重要的数据分析服务，它还有一个子服务叫做Amazon HealthLake Imaging，未来有可能成为PACS医疗影像归档和通信系统的基础，它要解决医疗场景下的特定问题。亚马逊云科技有非常多普适性的存储服务、分析服务，把这些普适性的服务放到医疗的场景下再做一些开发，就可以解决类似最短时间内查询的问题。

医疗行业要想继续发展，跟数据共享交换是分不开的。沿着数据的旅程，无论是自己存储、自己分析、未来自己去训练机器学习模型，到了机器学习模型，经常会发现数据永远是不够的，除了我们给客户有96个开放数据集之外，很多时候生态里的人希望数据共享。全球化的医疗公司和药企，有数据合规问题、有分公司数据权限管控的问题。但万变不离其宗，不能阻碍数据的使用。

Amazon DataZone帮药企跨部门去做细粒度的数据权限的管控，就是要找到平衡点，既把数据的权限管控好，同时鼓励使用数据去做分析。另外Amazon Clean Rooms希望能够让数据可用不可见，也就是说根本看不到原始数据，但是模型能够调用数据，能够让这个模型更健壮，能造福整个行业，这种想法和合作现在在行业里都在迭代。

本文由「Ben」原创出品， 转载或内容合作请点击 转载说明 ；违规转载必究。

寻求报道 。

本文图片来自：Pexels
'''

print(num_tokens(long_text))

8049


In [ ]:
response = openai.ChatCompletion.create(
    messages=[{'role': 'system', 'content':'为用户提供的文本生成摘要'}],
    model=GPT_MODEL,
)

可以看到这篇文章有 8049 个 token，直接发送给 GPT 会导致长度超限，返回错误。

现在我们来定义一个函数，将文章拆分成 token 更小的段落。

1. 将 long_text 按照换行符拆分成一个列表，这是为了尽量保持语义完整。
2. 对于列表中的每一项，首先检查其 token 数量：
   - 如果 token 数量超过了 max_tokens，则将其直接拆分成每一项不超过 max_token 的字符串列表，然后添加到 results；
   - 如果长度没有超过 max_tokens，则直接添加到 results。
3. 返回 results。

In [45]:
def split_long_text(long_text: str, max_tokens: int=MAX_TOKENS) -> list[str]:
    text_list = [ x for x in long_text.split("\n") if len(x.strip()) > 0 ]
    results = []

    for text in text_list:
        tokens = num_tokens(text)
        if tokens > max_tokens:
            for i in range(0, tokens, max_tokens):
                results.append(text[i:i+max_tokens])
        else:
            results.append(text)

    return results

splitted = split_long_text(long_text)
splitted

['36氪专访 | 亚马逊云科技顾凡：云计算、大数据和人工智能是加速医疗与生命科学行业数字化转型的重要途径',
 'Ben',
 '·2023-05-16 13:42',
 '关注',
 '加速推进创新成果从实验室到真实世界的落地',
 '“在1990年，需要十三年才能完成一个人基因测序，现在只需要一天就能测60个人，而每个人的测序成本降到百万分之一，而大规模的基因测序，需要调用数万台虚拟服务器同时进行。”',
 '“十年间，亚马逊云科技服务了全球超过4200个医疗及生命科学行业客户，包括辉瑞、拜尔、罗氏、默沙东、GE、飞利浦等头部企业。全球前十大药企中，九家都在使用亚马逊云科技。”',
 '“亚马逊云科技通过了全球百项安全合规认证，符合全球50+医疗生命科学合规要求。”',
 '“20 多年来，人工智能和机器学习一直是亚马逊关注的焦点。亚马逊云科技的使命是让各种技能水平的开发人员和各种规模的组织都有机会使用生成式AI进行创新。”',
 '4月27日，在“2023亚马逊云科技医疗与生命科学行业峰会”上，亚马逊云科技大中华区战略业务发展部总经理顾凡在演讲中围绕“数据、算力和体验”正在加速行业数字化创新，重点介绍了亚马逊云科技在全球以及中国医疗与生命科学行业的实践。',
 '在会后的座谈中，顾凡和亚马逊云科技大中华区医疗及生命科学行业总监黄庆春，共同与36氪及其他参会者进行了深入地交流。',
 '对于目前大热的AI大模型，顾凡认为：“我们既不应该去高估AIGC大模型的力量，但是更重要的是，不能低估AIGC大模型会重塑大量行业客户的体验。过去20多年的人工智能和机器学习一直都是亚马逊关注的焦点，亚马逊无论是对外的服务，还是对内部的运营，到处都可以看到人工智能和机器学习的身影。在生成式AI方面，我们的使命就是，要让各种技能水平的开发人员和各种组织都有机会使用生成式AI进行创新。”',
 '亚马逊云科技长期关注生物制药、基因测序和医疗保健领域的客户需求，发现三大共同挑战：',
 '数据挑战：数据量的剧增带来洞察力挑战，科学家发现，自然界里可能成为药物的分子个数有10的60次方，而太阳系的原子个数才是10的50次方；',
 '算力挑战：在1990年，需要十三年才能完成一个人基因测序，现在只需要一天就能测60个人，而每个人的测序成本降到百万分之一，而大规模的基因测序，需要调用数万

这样我们就得到了一个每一项都不超限的字符串列表，可以让 GPT 依次生成摘要了。

但是我们也发现，这个列表里有很多项的长度非常短。理想的情况是我们列表里每一项的长度都尽可能地接近上限，这样一来可以减少请求次数（省钱），二来过短的段落也没有摘要的必要。

我们再写一个函数，用来合并列表里的项目。

In [46]:
def join_short_strings(texts: list[str], max_tokens: int=MAX_TOKENS) -> list[str]:
    results = []
    buffer = ''
    for text in texts:
        if len(buffer) == 0:
            buffer = text
        else:
            if num_tokens(f'{buffer}\n{text}') <= max_tokens:
                buffer = f'{buffer}\n{text}'
            else:
                results.append(buffer)
                buffer = text
    return results

joined = join_short_strings(splitted)
joined

['36氪专访 | 亚马逊云科技顾凡：云计算、大数据和人工智能是加速医疗与生命科学行业数字化转型的重要途径\nBen\n·2023-05-16 13:42\n关注\n加速推进创新成果从实验室到真实世界的落地\n“在1990年，需要十三年才能完成一个人基因测序，现在只需要一天就能测60个人，而每个人的测序成本降到百万分之一，而大规模的基因测序，需要调用数万台虚拟服务器同时进行。”\n“十年间，亚马逊云科技服务了全球超过4200个医疗及生命科学行业客户，包括辉瑞、拜尔、罗氏、默沙东、GE、飞利浦等头部企业。全球前十大药企中，九家都在使用亚马逊云科技。”\n“亚马逊云科技通过了全球百项安全合规认证，符合全球50+医疗生命科学合规要求。”\n“20 多年来，人工智能和机器学习一直是亚马逊关注的焦点。亚马逊云科技的使命是让各种技能水平的开发人员和各种规模的组织都有机会使用生成式AI进行创新。”\n4月27日，在“2023亚马逊云科技医疗与生命科学行业峰会”上，亚马逊云科技大中华区战略业务发展部总经理顾凡在演讲中围绕“数据、算力和体验”正在加速行业数字化创新，重点介绍了亚马逊云科技在全球以及中国医疗与生命科学行业的实践。\n在会后的座谈中，顾凡和亚马逊云科技大中华区医疗及生命科学行业总监黄庆春，共同与36氪及其他参会者进行了深入地交流。\n对于目前大热的AI大模型，顾凡认为：“我们既不应该去高估AIGC大模型的力量，但是更重要的是，不能低估AIGC大模型会重塑大量行业客户的体验。过去20多年的人工智能和机器学习一直都是亚马逊关注的焦点，亚马逊无论是对外的服务，还是对内部的运营，到处都可以看到人工智能和机器学习的身影。在生成式AI方面，我们的使命就是，要让各种技能水平的开发人员和各种组织都有机会使用生成式AI进行创新。”\n亚马逊云科技长期关注生物制药、基因测序和医疗保健领域的客户需求，发现三大共同挑战：\n数据挑战：数据量的剧增带来洞察力挑战，科学家发现，自然界里可能成为药物的分子个数有10的60次方，而太阳系的原子个数才是10的50次方；\n算力挑战：在1990年，需要十三年才能完成一个人基因测序，现在只需要一天就能测60个人，而每个人的测序成本降到百万分之一，而大规模的基因测序，需要调用数万台虚拟服务器同时进行；\n体验挑战：医院用的医疗信息软件有近100种，常用的就有约2

现在我们就得到了一个合适的列表，其中每一项的 token 数量都不超过我们的限制，且列表长度尽可能小。

## 要求 GPT 生成摘要

接下来我们写几个函数，用来要求 GPT 生成文章摘要。

In [55]:
def summarize(text: str, model: str=GPT_MODEL) -> str:
    system_message = {
        'role': 'system',
        'content': '为用户提供的文本生成摘要'
    }
    human_message = {
        'role': 'user',
        'content': text
    }
    response = openai.ChatCompletion.create(
        messages=[system_message, human_message],
        model=model,
    )
    return response['choices'][0]['message']['content']

def summarize_all(texts: list[str], model: str=GPT_MODEL) -> str:
    summary = ''
    for text in texts:
        summary += summarize(text, model=model)
    return summarize(summary)

def summarize_long_text(long_text: str, model: str=GPT_MODEL) -> str:
    text_list = split_long_text(long_text)
    text_list = join_short_strings(text_list)
    return summarize_all(text_list)

summarize_long_text(long_text)

'亚马逊云科技在“2023亚马逊云科技医疗与生命科学行业峰会”上介绍了其在医疗与生命科学行业数字化转型中的重要作用。亚马逊云科技提供了云计算、大数据和人工智能等方面的支持，帮助客户解决数据、算力和体验挑战。此外，亚马逊云科技也提供行业解决方案，包括存储、分析和阐述基因组学、转录组学和其他组学数据的托管服务，医疗成像服务，以及自然语言处理服务等。亚马逊云科技还开放了96种生命科学公开数据集，助力客户加速新药研发。AI技术的应用在高性能计算、AI辅助药物研发以及量子计算等方向上解决问题，同时亚马逊云科技提供的Amazon SageMaker平台降低了AI使用门槛，同时打包了一些常用算法和技术解决方案帮助客户快速实验并提高效率。'

可以看到，我们成功为这篇文章生成了摘要，并没有丢失太多信息。

## 接下来做什么

接下来你还可以继续迭代这段代码，当生成的摘要最后仍然超限的时候继续拆分，以及加入适当的异常处理机制。但是软件工程层面的优化就不在本文档的讨论范围之内了。

理解了如何让 OpenAI 处理长文本，那么除了生成摘要，像翻译、归类、审核、结构化等任务也可以交给它来处理了。